# Chest-prediction

## PRELIMINARY ACTION !

⚠️ Please go to ➤ https://drive.google.com/file/d/1lLrHbpUQE-Kd-jZ68Uk7SFwawbzqf6Av/view?usp=drive_link

and download the dataset.

Put the zip file into your "*raw_data*" folder

## data loading

In [1]:
import numpy as np
import pandas as pd
import pdb

import os
from pathlib import Path
from PIL import Image

### Loading images

In [2]:
PROJECT_NAME = "chest-predictor" #to be adapted depending on the name of the Project Name in your system
NUMBER_OF_IMAGES = 1000 # nbr of images to be loaded or 'full' to load the entire dataset (+100k images)

In [3]:
USERNAME = os.environ.get('USER')

In [4]:
LOCAL_DATA_PATH = Path(f"/Users/{USERNAME}/code/sachamagier/{PROJECT_NAME}/raw_data/resized_dataset")

In [5]:
print(f"LOCAL_DATA_PATH: {LOCAL_DATA_PATH}")

LOCAL_DATA_PATH: /Users/rvnmll/code/sachamagier/chest-predictor/raw_data/resized_dataset


In [6]:
def loading_data():
    """This function either get all the images if the user set NUMBER_OF_IMAGES
    to 'full' or the number of imgaes otherwise """

    images_data = []

    # Define the path to the folder
    folder_path = f'../raw_data/resized_dataset/images/set_full/'


    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)

    # Filter the list to only include image files
    image_files = [f for f in file_list if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]


    # Loop through the first NUMBER_OF_IMAGES
    for i, image_file in enumerate(image_files):

        # Stop the loop after NUMBER_OF_IMAGES iterations
        if i == NUMBER_OF_IMAGES:
            break

        # Open the image file
        with Image.open(folder_path + image_file) as image:
            # Add the image to the list
            images_data.append((image_file, np.array(image)))

    return images_data

In [11]:
import urllib.request
import zipfile
import shutil


if LOCAL_DATA_PATH.is_dir():
    print("Load local data...")
    # loading data into data
    images_data = loading_data()
else:
    print("Unziping file and loading the data...")

    output_path = "../raw_data/resized_dataset.zip"
    # unzip the file
    with zipfile.ZipFile(output_path, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            zip_ref.extract(file_info, "../raw_data/")

    if Path("../raw_data/__MACOSX").is_dir():
        # remove the __MACOSX folder if it exists
        shutil.rmtree("../raw_data/__MACOSX")

    # remove the zip file
    os.remove(output_path)
    images_data = loading_data()

print("data loaded.")

Unziping file and loading the data...
data loaded.


In [13]:
# Create a dataframe from the list of images and their indices
images_df = pd.DataFrame(images_data, columns=['Image Index', 'image'])

# Set the index of the dataframe to the 'Image Index' column
images_df = images_df.set_index('Image Index').sort_index(ascending=True)

In [14]:
images_df.shape

(1000, 1)

In [15]:
images_df['image'][0].shape

/tmp/ipykernel_20441/4042123707.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  images_df['image'][0].shape


(256, 256)

### Loading labels data

In [16]:
labels_df = pd.read_csv('../raw_data/resized_dataset/Data_Entry_2017.csv')

In [17]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [18]:
unique_lab = labels_df['Finding Labels'].unique()

In [19]:
len(unique_lab)

836

### Merging Images with labels and creating a new DF

In [20]:
# Merge the image_df and labels_df dataframes on the 'Image Index' column
merged_df = pd.merge(images_df, labels_df[['Image Index', 'Finding Labels']], left_index=True, right_on='Image Index', how='inner')

# Rename the 'Finding Labels' column to 'labels'
merged_df = merged_df.rename(columns={'Finding Labels': 'labels'})

# Set the index of the dataframe to the 'Image Index' column
merged_df = merged_df.set_index('Image Index').sort_index(ascending=True)

In [21]:
merged_df.head()

,image,labels
Image Index,,
00000006_000.png,"[[45, 43, 38, 34, 30, 27, 24, 21, 19, 17, 16, ...",No Finding
00000013_031.png,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",Emphysema|Mass
00000032_005.png,"[[6, 7, 6, 6, 7, 7, 6, 6, 6, 7, 7, 7, 6, 6, 6,...",No Finding
00000032_026.png,"[[15, 14, 12, 11, 10, 9, 9, 8, 6, 89, 130, 125...",Infiltration
00000041_001.png,"[[2, 2, 1, 11, 60, 36, 32, 32, 30, 29, 29, 27,...",Emphysema


### droping the rows with images of shape (256, 256, 4)

In [22]:
# Find the images with shape (256, 256, 4)
images_with_shape_4 = [img for img in merged_df['image'] if np.shape(img) == (256, 256, 4)]

# Print the number of images with shape (256, 256, 4)
print(len(images_with_shape_4))


4


In [23]:
# Find the indices of the images with shape (256, 256, 4)
indices_to_drop = merged_df[merged_df['image'].apply(lambda x: np.shape(x) == (256, 256, 4))].index

# Drop the rows with the images with shape (256, 256, 4)
merged_df = merged_df.drop(indices_to_drop)

In [24]:
# Find the images with shape (256, 256, 4)
images_with_shape_4 = [img for img in merged_df['image'] if np.shape(img) == (256, 256, 4)]

# Print the number of images with shape (256, 256, 4)
print(len(images_with_shape_4))

0


### Encoding labels

In [25]:
# Define the list of labels
labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema',
           'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
           'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding']

# Create a new dataframe with one-hot encoded columns for the labels
one_hot_df = merged_df['labels'].str.get_dummies(sep='|')

# Concatenate the one-hot encoded dataframe with the original dataframe
merged_df = pd.concat([merged_df, one_hot_df], axis=1)


In [26]:
merged_df.columns

Index(['image', 'labels', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia',
       'Pneumothorax'],
      dtype='object')

## Creating 'X' and 'y'

In [27]:
y = one_hot_df

In [28]:
X = merged_df['image']

In [29]:
X.info(memory_usage='deep')

<class 'pandas.core.series.Series'>
Index: 996 entries, 00000006_000.png to 00030775_000.png
Series name: image
Non-Null Count  Dtype 
--------------  ----- 
996 non-null    object
dtypes: object(1)
memory usage: 62.4 MB


In [30]:
X = np.array([np.reshape(img, (256, 256, 1)) for img in X])

In [31]:
X.shape

(996, 256, 256, 1)

## Creating a simple model

In [42]:
#Imports
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.callbacks import EarlyStopping

In [43]:
##### Splitting X and y in train/val/test #####

# First split: split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Second split: split training data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [44]:
#Image parameters
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 32
NUM_CLASSES = 15

In [50]:
# Define a simple model
def simple_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,1)))

    model.add(Conv2D(16, kernel_size=10, activation='relu'))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=8, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=6, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [51]:
s_model = simple_model()
es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights=True)
s_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 256, 256, 1)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 247, 247, 16)      1616      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 82, 82, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 75, 75, 32)        32800     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 25, 25, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 20, 20, 32)       

In [52]:
# Train the model
history = s_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=BATCH_SIZE,
    callbacks=[es])

Epoch 1/10
20/20 [==============================] - 5s 198ms/step - loss: 3.6320 - accuracy: 0.4827 - val_loss: 9.4092 - val_accuracy: 0.5125
Epoch 2/10
20/20 [==============================] - 4s 190ms/step - loss: 98.3148 - accuracy: 0.3396 - val_loss: 343.7252 - val_accuracy: 0.0875
Epoch 3/10
20/20 [==============================] - 5s 232ms/step - loss: 978.0417 - accuracy: 0.2720 - val_loss: 3175.0625 - val_accuracy: 0.1562
Epoch 4/10
20/20 [==============================] - 4s 213ms/step - loss: 7157.8491 - accuracy: 0.3286 - val_loss: 17955.2500 - val_accuracy: 0.0125
Epoch 5/10
20/20 [==============================] - 4s 204ms/step - loss: 41091.9609 - accuracy: 0.2704 - val_loss: 100497.4844 - val_accuracy: 0.0188
Epoch 6/10
20/20 [==============================] - 5s 259ms/step - loss: 191437.3750 - accuracy: 0.2909 - val_loss: 396619.7812 - val_accuracy: 0.5125


In [53]:
test_loss, test_accuracy = s_model.evaluate(X_test, y_test)
val_loss, val_accuracy = s_model.evaluate(X_val, y_val)

5/5 [==============================] - 0s 62ms/step - loss: 9.4092 - accuracy: 0.5125


In [54]:
print('val accuracy:', val_accuracy)
print('test accuracy:', test_accuracy)

val accuracy: 0.512499988079071
test accuracy: 0.5149999856948853


In [56]:
y_pred = s_model.predict(X_test)

7/7 [==============================] - 0s 54ms/step


In [62]:
y_pred[0]

array([1.        , 0.9999852 , 0.9999999 , 0.99999714, 1.        ,
       0.9962546 , 0.9355537 , 0.00861453, 1.        , 1.        ,
       1.        , 0.9999995 , 0.99999994, 0.10169847, 1.        ],
      dtype=float32)

In [63]:
y.head()

,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
Image Index,,,,,,,,,,,,,,,
00000006_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000013_031.png,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
00000032_005.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000032_026.png,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
00000041_001.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [55]:
from tensorflow.keras.models import save_model

models_dir = '../models'

# Ensure that the models directory exists
os.makedirs(models_dir, exist_ok=True)


# Save the trained model to the models directory
save_model(s_model, os.path.join(models_dir, 'simple_model_correct.h5'))

/tmp/ipykernel_20441/3205834225.py:10: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(s_model, os.path.join(models_dir, 'simple_model_correct.h5'))
